# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [39]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [40]:
# Create a temporary workspace
import tempfile


SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

Session workspace created at: /tmp/tmp3lvgaiml


In [41]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

Unpacking zipped data...
Zipped dataset unpacked to /tmp/tmp3lvgaiml/duckietown_object_detection_dataset
#Files in train/images: 804
#Files in train/labels: 804
#Files in val/images: 202
#Files in val/labels: 202


In [62]:
# change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision
!pip3 install torch==1.13.0 torchvision
#!pip3 install torch==1.13.0 torchvision==0.14.0

PWD: /tmp/tmp3lvgaiml
  Using cached torch-1.13.0-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached torchvision-0.20.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached torchvision-0.20.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached torchvision-0.19.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached torchvision-0.19.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached torchvision-0.18.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torchvision-0.18.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a wh

# Next, we will clone Yolov5

In [44]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17372, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 17372 (delta 42), reused 20 (delta 20), pack-reused 17313 (from 3)
Receiving objects: 100% (17372/17372), 16.24 MiB | 14.28 MiB/s, done.
Resolving deltas: 100% (11904/11904), done.
Note: switching to '915bbf294bb74c859f0b41f1c23bc395014ea679'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



# We now inform the training process of the format and location of our dataset

In [45]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Writing ./yolov5/data/duckietown.yaml


# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [56]:
!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 32 --epochs 10 --data duckietown.yaml --weights yolov5n.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.4 MB/s eta 0:00:00

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/tmp/tmp3lvgaiml/yolov5/train.py", line 42, in <module>
    import val as validate  # for end-of-epoch mAP
  File "/tmp/tmp3lvgaiml/yolov5/val.py", line 38, in <module>
    from models.common import DetectMultiBackend
  File "/tmp/tmp3lvgaiml/yolov5/models/common.py", line 29, in <module>
    from utils.dataloaders import exif_transpose, letterbox
  File "/tmp/tmp3lvgaiml/yolov5/utils/dataloaders.py", line 25, in <module>
    import to

In [57]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

Latest exp is exp3
cp: cannot stat 'yolov5/runs/train/exp3/weights/best.pt': No such file or directory

Marked the model from the latest run (exp3) as yolov5/best.pt.


# Next, we can upload your model to Duckietown's cloud!

We will need our token to access our personal cloud space.

In [58]:
# TODO: Fill in the duckietown token here
YOUR_DT_TOKEN = "dt1-DHcEZkb4F7AbsdjUR3SvZud36MzrK5MzKAbLuoVeyvm5NYuJ-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfbh2S93NeJvE8jnsM7va9CsaFWCeHabJ8p"

Then, we chose the location of the trained model on disk and its name once uploaded to our cloud space. You should not change these values, or the robots will not be able to find the model to download.

In [59]:
import sys
sys.path.insert(0, './yolov5')

# DO NOT CHANGE THESE
model_name = "yolov5n"
model_local_path = "./yolov5/best.pt"
model_remote_path = f"courses/mooc/objdet/data/nn_models/{model_name}.pt"

# install DCSS client
!pip3 install dt-data-api

We now open a pointer to our cloud space and upload the model.

In [60]:
import torch
from dt_data_api import DataClient, Storage

# open a pointer to our personal duckietown cloud space
client = DataClient(YOUR_DT_TOKEN)
storage = client.storage("user")

# upload model
upload = storage.upload(model_local_path, model_remote_path)
upload.join()

ValueError: The file /tmp/tmp3lvgaiml/yolov5/best.pt does not exist.

# Done!

We're done training! You can now close this tab and go back to the `Training` notebook